# GroupBy and Aggregate Functions

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Aggs").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/26 02:30:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv("data/sales_info.csv", inferSchema=True, header=True)

In [3]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [4]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [5]:
# groupBy returns GroupedData object (like python .groupby)
df.groupBy("Company")

In [8]:
# Calling .mean method on groupeddata object returns a datafame
df.groupBy("Company").min().show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [9]:
df.groupBy("Company").count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.GroupedData.html

In [14]:
# Can also call agg functions without grouping first (e.g. to get column totals)
df.agg({"Sales":"max"}).show(), df.agg({"Sales":"sum"}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



(None, None)

In [15]:
group_data = df.groupBy("Company")

In [18]:
group_data.agg({"Sales":"max"}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [19]:
# Import useful functions from pyspark
from pyspark.sql.functions import countDistinct, avg, stddev

In [20]:
# Pass a pyspark.sql function to df.select
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [22]:
# Can provide an alias for output column
df.select(stddev('Sales').alias("StdDev Sales")).show()

+------------------+
|      StdDev Sales|
+------------------+
|250.08742410799007|
+------------------+



In [23]:
# Can format numbers with pyspark.sql function format_number()
from pyspark.sql.functions import format_number

In [24]:
sales_std = df.select(stddev('Sales').alias("StdDev Sales"))

In [27]:
sales_std.select(format_number("StdDev Sales", 2).alias("StdDev Sales")).show()

+------------+
|StdDev Sales|
+------------+
|      250.09|
+------------+



In [30]:
# Ordering and Sorting
df.orderBy("Sales", ascending=False).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [31]:
df.orderBy(df["Sales"].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

